# 06 - Pit Loss Model

## Overview
Compute circuit-average pit losses.

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
from f1ts import config, io_flat, models_pitloss

## Load

In [2]:
raw_dir = config.paths()['data_raw']
pitstops_files = list(raw_dir.glob('*_pitstops.csv'))
all_pitstops = []
for pf in pitstops_files:
    if pf.stat().st_size > 0:
        all_pitstops.append(pd.read_csv(pf))
pitstops = pd.concat(all_pitstops, ignore_index=True) if all_pitstops else pd.DataFrame()
sessions = io_flat.read_csv(raw_dir / 'sessions.csv')

✓ Loaded sessions.csv: 10 rows, 6 cols


## Transform

In [3]:
if len(pitstops) > 0:
    circuit_avg = models_pitloss.compute_circuit_averages(pitstops, sessions)
    print(circuit_avg)
else:
    print('No pit stop data available')

          circuit_name  pit_loss_s  pit_loss_std  n_stops
0  Austrian Grand Prix      77.718           NaN        1


## Save

In [4]:
paths = config.paths()
lookups_dir = paths['data_lookups']

# Save circuit averages to computed lookup if available
try:
    if 'circuit_avg' in locals() and isinstance(circuit_avg, pd.DataFrame) and len(circuit_avg) > 0:
        out_path = lookups_dir / 'pitloss_computed.csv'
        circuit_avg[['circuit_name', 'pit_loss_s']].to_csv(out_path, index=False)
        print(f"✓ Saved pit loss lookup to {out_path}")
    else:
        print('No circuit averages to save.')
except Exception as e:
    print(f"Failed to save pit loss lookup: {e}")

✓ Saved pit loss lookup to /workspaces/f1-track-strategy/data/lookups/pitloss_computed.csv


## Repro Notes

- Computed average pit loss by circuit
- Used simple averaging baseline